In [ ]:
#| default_exp main

In [ ]:
#| export
from uuid import uuid4
from typing import Callable
from dataclasses import dataclass
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
import fastcore.all as fc
from meme_games.user import *
from meme_games.init import *
from meme_games.lobby import *
from fastlite import *

In [ ]:
#| export
db = init_db()
user_manager, lobby_manager = init_services(db)
bwares = [user_beforeware(user_manager)]
app, rt = fast_app(pico=False, before=bwares, hdrs=hdrs, exts='ws', bodykw={'hx-boost': 'true'})

In [ ]:
#| export
import asyncio

async def send(u: LobbyMember, fn, *args):
    try: await u.ws(fn(*args))
    except: u.ws = None

async def notify_all(lobby:Lobby, fn: Callable, *args):
    tasks = [send(u, fn, lobby, *args) for u in lobby.members.values() if u.is_connected]
    await asyncio.gather(*tasks)

In [ ]:
#| export
@rt("/{lobby_id}")
def get(req: Request, lobby_id: str=None):
    u: User = req.state.user
    if lobby:=lobby_manager.get_lobby(lobby_id or ''): 
        req.session['lobby_id'] = lobby.id
        return Titled(f'Lobby {lobby.id}', 
                     Members(lobby),
                     Div(hx_ext='ws', ws_connect='/game'),
                     NameSetting())
    
    lobby = lobby_manager.create_lobby(u)
    return Redirect(f"/{lobby.id}")

def ws_fn(connected=True):
    async def user_joined(sess, send):
        u = user_manager.get_or_create(sess)
        lobby = lobby_manager.get_lobby(sess.get('lobby_id'))
        if not lobby: return
        lobby.get_or_create_member(u, send if connected else None)
        await notify_all(lobby, Members)
    return user_joined
    

@app.ws('/game', conn=ws_fn(), disconn=ws_fn(connected=False))
async def ws(send): pass


In [ ]:
#| export
@rt('/name')
async def put(req: Request, name:str):
    u: User = req.state.user
    u.name = name
    user_manager.update(u)
    lobby: Lobby = lobby_manager.get_lobby(req.session.get("lobby_id"))
    if not lobby: return
    def update(lobby): return UserName(u)
    await notify_all(lobby, update)

In [ ]:
#| export
server = JupyUvi(app)

session: {'uid': '065f3f83-599e-48b1-aefe-eff14ba45b7e', 'lobby_id': 'r441z'}
lobby is Lobby(id='r441z', host=LobbyMember(user=User(uid='065f3f83-599e-48b1-aefe-eff14ba45b7e', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 53, 27, 698969)), members={'065f3f83-599e-48b1-aefe-eff14ba45b7e': LobbyMember(user=User(uid='065f3f83-599e-48b1-aefe-eff14ba45b7e', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 53, 27, 698969)), '17f02c83-8532-4d16-9138-1f8411597b6f': LobbyMember(user=User(uid='17f02c83-8532-4d16-9138-1f8411597b6f', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 53, 50, 23534))}, last_active=datetime.datetime(2025, 1, 4, 22, 53, 27, 698994))
session: {'uid': '17f02c83-8532-4d16-9138-1f8411597b6f', 'lo

In [ ]:
lobby_manager.get_lobby("8gz79")

Lobby(id='8gz79', host=LobbyMember(user=User(uid='17f02c83-8532-4d16-9138-1f8411597b6f', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 46, 45, 679772)), members={'17f02c83-8532-4d16-9138-1f8411597b6f': LobbyMember(user=User(uid='17f02c83-8532-4d16-9138-1f8411597b6f', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 46, 45, 679772)), '065f3f83-599e-48b1-aefe-eff14ba45b7e': LobbyMember(user=User(uid='065f3f83-599e-48b1-aefe-eff14ba45b7e', name='null'), ws=functools.partial(<function _send_ws>, <starlette.websockets.WebSocket object>), joined_at=datetime.datetime(2025, 1, 4, 22, 46, 50, 689516))}, last_active=datetime.datetime(2025, 1, 4, 22, 46, 45, 679785))

In [ ]:
server.stop()